In [ ]:
# !pip install --q langchain-core
# !pip install --q langchain-community
# !pip install --q langchain_google_genai
# !pip install --q crewai[tools]

In [ ]:
from dotenv import load_dotenv
print("Environment variables are loaded:", load_dotenv())

## **Database Preparation**
The data used in this project is a snapshot of the modern job market on the role of AI from [Kaggle](https://www.kaggle.com/datasets/uom190346a/ai-powered-job-market-insights?resource=download).

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("data/ai_job_market_insights.csv")

df.head()

In [ ]:
df.info()

In [ ]:
import sqlite3

# Create a SQLite database and write the data to a table
connection = sqlite3.connect("job_insights.db")
df.to_sql(name="job_insights", con=connection, if_exists='replace', index=False)

## **Implementing Tools for SQL Operations**
Define various tools to interact with the SQLite database. These tools will include functionalities to list tables, get table schemas, execute SQL queries, and check SQL queries for correctness.

In [ ]:
import os
from langchain_google_genai.llms import GoogleGenerativeAI
# from google.colab import userdata

# Define llm
llm = GoogleGenerativeAI(model="gemini-pro",
                         google_api_key=os.getenv('GOOGLE_API_KEY'))

In [ ]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)
from langchain_community.utilities.sql_database import SQLDatabase
from crewai_tools import tool


# Load the database
db = SQLDatabase.from_uri("sqlite:///job_insights.db")

# Define the tools
@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

list_tables.run()

In [ ]:
@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling 'list_table' first.
    Example Input: table1, table2, table3
    """
    tool = InfoSQLDatabaseTool(db=db)
    return tool.invoke(tables)

print(tables_schema.run("job_insights"))

In [ ]:
@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result of the query."""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

execute_sql.run("SELECT DISTINCT Industry FROM job_insights WHERE AI_Adoption_Level = 'High'")

In [ ]:
@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double-check if your query is correct before executing
    it with 'execute_sql'. Always use this tool before executing a query with 'execute_sql'.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

check_sql.run("SELECT * WHERE Salary_USD < 100000 LIMIT 5 table = job_insights")

In [ ]:
from textwrap import dedent
from crewai import Agent, Task, Crew, Process

sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
)

In [ ]:
data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have more than 10 years of analyzing datasets using Python and are an expert at it.
        Your work is always based on the provided data and is clear,
        easy-to-understand, and straightforward. You have attention to detail
        and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)

In [ ]:
report_generator = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing is well known for its clearness and effectiveness in delivering insights.
        You always summarize long texts into bullet points that contain the most important details.
    """
    ),
    llm=llm,
    allow_delegation=False,
)

## **Creating Tasks and Crew**
We defined tasks for each agent and create a Crew to manage the process. Each task corresponds to a specific step in our workflow, from data extraction to analysis and report generation.

In [ ]:
extract_data = Task(
    description="Extract data that is required for the query {query}",
    expected_output="Database result for the query",
    agent=sql_dev,
)

In [ ]:
analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

In [ ]:
write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis.
        You always write reports in bullet points of the key findings.
        The report must be less than 100 words.
    """
    ),
    expected_output="Markdown report",
    agent=report_generator,
    context=[analyze_data],
)

In [ ]:
crew =  Crew(
    agents=[sql_dev, data_analyst, report_generator],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,
    output_log_file="crew.log",
)

## **Executing the Process**
Finally, we execute the process by providing the query input to the crew. This input will be processed sequentially by each agent according to their defined tasks.

In [ ]:
inputs = {
    "query": "How is the salary in USD of a Data Scientist based on the company's AI adoption level?"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
print(result)

In [ ]:
inputs = {
    "query": "Which industry is leading with its AI adoption?"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
print(result)

In [ ]:
inputs = {
    "query": "What are the top 5 desired jobs that pay more than 100000 USD?"
}

result = crew.kickoff(inputs=inputs)

In [ ]:
print(result)